# Generating User Facts

In [1]:
import pandas as pd
import random
import os
import json

In [7]:
users = list(range(1, 25))
user_facts = []
food_facts = []
pos_pairs = []
neg_pairs = []

In [8]:
def partition(arr, num_pos, num_neg):
    # Shuffle the array
    random.shuffle(arr)
    
    # Determine the size of each partition
    partition_size = len(arr) // 3
    
    # Create the three partitions
    pos = arr[:num_pos]
    neg = arr[num_pos: num_pos + num_neg]
    remaining = arr[num_pos + num_neg:]
    
    return pos, neg, remaining

In [9]:
dairy_opinions = partition(users, 2, 2)
meat_opinions = partition(users, 2, 2)
nut_opinions = partition(users, 2, 2)

In [10]:
for feature_name, opinions in [('dairy',dairy_opinions),
                               ('meat', meat_opinions), 
                               ('nuts', nut_opinions)]:
    positive, negative, _ = opinions
    for opinion, type_str in [(positive,'positive'),
                           (negative, 'negative')]:
        for user in opinion:
            user_facts.append(f'preference(user_{user}, {type_str}_{feature_name}).')
[print(user_fact) for user_fact in user_facts]
print(dairy_opinions)
print(nut_opinions)
print(meat_opinions)


preference(user_2, positive_dairy).
preference(user_24, positive_dairy).
preference(user_18, negative_dairy).
preference(user_5, negative_dairy).
preference(user_21, positive_meat).
preference(user_14, positive_meat).
preference(user_10, negative_meat).
preference(user_3, negative_meat).
preference(user_14, positive_nuts).
preference(user_4, positive_nuts).
preference(user_9, negative_nuts).
preference(user_15, negative_nuts).
([2, 24], [18, 5], [6, 22, 11, 10, 23, 21, 16, 3, 8, 14, 17, 20, 19, 1, 12, 4, 9, 13, 7, 15])
([14, 4], [9, 15], [21, 3, 24, 2, 22, 11, 6, 10, 20, 16, 7, 8, 1, 5, 17, 23, 12, 18, 19, 13])
([21, 14], [10, 3], [7, 19, 6, 4, 2, 23, 5, 16, 12, 24, 20, 1, 13, 18, 15, 17, 8, 22, 11, 9])


# Generating Food Facts

In [5]:
# Loading data
beverages = json.load(open('../items_data/beverages.json'))
beverages = beverages['beverage_ids']

mcdonalds = json.load(open('../items_data/mcdonalds.json'))
mcdonalds = mcdonalds['recipe-ids']

taco_bell = json.load(open('../items_data/taco_bell.json'))
taco_bell = taco_bell['recipe-ids']

treat_data=json.load(open('../items_data/recipe_repn.json'))
treat_data = treat_data['recipe-ids']

In [6]:
for data, is_bev in ((beverages, True), (mcdonalds, False), (taco_bell, False), (treat_data, False)):
    for key, item_info in data.items():
        if item_info['hasNuts']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_nuts).')
            else:
                food_facts.append(f'item(food_{key}, has_nuts).')
        if item_info['hasMeat']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_meat).')
            else:
                food_facts.append(f'item(food_{key}, has_meat).')
        if item_info['hasDairy']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_dairy).')
            else:
                food_facts.append(f'item(food_{key}, has_dairy).')

# Generating positive/negative pairs

In [7]:
pos_dairy, neg_dairy, _ = dairy_opinions
pos_meat, neg_meat, _ = meat_opinions
pos_nuts, neg_nuts, _ = nut_opinions

pos_scores = 0
neg_scores = 0
zero_scores = 0

for user in users:
    for data, is_bev in ((beverages, True), (mcdonalds, False), (taco_bell, False), (treat_data, False)):
        for key, item_info in data.items():
            score = 0
            if ((item_info['hasNuts'] and user in neg_nuts) or
                 (item_info['hasDairy'] and user in neg_dairy) or
                 (item_info['hasMeat'] and user in neg_meat)):
                if is_bev:
                    neg_pairs.append(f'recommendation(user_{user}, bev_{key}).')
                else:
                    neg_pairs.append(f'recommendation(user_{user}, food_{key}).')
            else:
                if is_bev:
                    pos_pairs.append(f'recommendation(user_{user}, bev_{key}).')
                else:
                    pos_pairs.append(f'recommendation(user_{user}, food_{key}).')

# Splitting Train/Test

In [8]:
def split_train_test(array, per_train=0.8):
    # shuffle the array
    random.shuffle(array)

    # Determine the split index
    split_index = int(per_train * len(array))
    
    # Split the array into train test
    train = array[:split_index]
    test = array[split_index:]
    return train, test

In [9]:
user_train, user_test = split_train_test(user_facts) 
food_train, food_test = split_train_test(food_facts) 
train_pos, test_pos = split_train_test(pos_pairs) 
train_neg, test_neg = split_train_test(neg_pairs) 

In [10]:
train_facts = user_train + food_train
test_facts = user_test + food_test

In [12]:
for contents, file_name in [(train_facts, 'train_facts.txt'),
                            (train_neg, 'train_neg.txt'),
                            (train_pos, 'train_pos.txt'),
                            (test_facts, 'test_facts.txt'),
                            (test_neg, 'test_neg.txt'),
                            (test_pos, 'test_pos.txt')]:
    with open(f'../boosted_bandit/{file_name}', 'w') as file:
        contents = [(line + '\n') for line in contents]
        file.writelines(contents)

In [2]:
with open('../boosted_bandit/user_facts.txt', 'w') as file:
    contents = [(line + '\n') for line in user_facts]
    file.writelines(contents)

NameError: name 'user_facts' is not defined